In [38]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

# Fetch dataset
heart_disease = fetch_ucirepo(id=45)

# Data (as pandas dataframes)
X = heart_disease.data.features
y = heart_disease.data.targets

# Display the first few rows of the full dataset
print(X.head())
print(y.head())

# add random noisy columns to the data
# np.random.seed(42)
# for i in range(10):
#     X[f'noise_{i}'] = np.random.normal(0, 1, len(X))
    
# # introduce redundant columns
# X['redundant_1'] = X['age'] + X['trestbps']
# X['redundant_2'] = X['chol'] + X['thalach']
# X['redundant_3'] = X['oldpeak'] + X['slope']

# # introduce columns with skewed distributions
# X['skewed_1'] = np.log(X['age'])
# X['skewed_2'] = np.sqrt(X['trestbps'])
# X['skewed_3'] = np.cbrt(X['chol'])

# # introduce columns with skewed distributions (synthetic)
# X['skewed_4'] = np.log(X['thalach'])
# X['skewed_5'] = np.sqrt(X['oldpeak'])
# X['skewed_6'] = np.cbrt(X['slope'])

# # Add a 'usubjid' column with a combination of string and numbers
# np.random.seed(42)
# usubjid = ['ID' + str(i) if i % 2 == 0 else 'ID' + str(i) + 'A' for i in range(1, len(X) + 1)]
# X.insert(0,'usubjid',usubjid)
# y.insert(0,'usubjid',usubjid)

# # Add duplicate rows
# X = pd.concat([X, X.sample(frac=0.1, random_state=42)], ignore_index=True)
# y = pd.concat([y, y.sample(frac=0.1, random_state=42)], ignore_index=True)

# # Introduce outliers
# num_outliers = int(0.05 * len(X))
# outlier_indices = np.random.choice(X.index, num_outliers, replace=False)
# X.loc[outlier_indices, X.columns[:-1]] = X.loc[outlier_indices, X.columns[:-1]] * 10

# # Replace 'sex' column values with inconsistent string conventions
# female_values = ['F', 'Female', 'female', 'feMAle']
# male_values = ['M', 'm', 'male', 'Male', 'mALe']
# X['sex'] = X['sex'].apply(lambda x: np.random.choice(female_values) if x == 1 else np.random.choice(male_values))

# # Add a 'stresslvl' column with values from ['ls1','ls2','ls3','ls4','ls5'] with the majority of entries 'ls1' or 'ls2'
stresslvl_values = ['ls1', 'ls2', 'ls3', 'ls4', 'ls5']
stresslvl_probs = [0.4, 0.4, 0.1, 0.05, 0.05]
X['stresslvl'] = np.random.choice(stresslvl_values, len(X), p=stresslvl_probs)

# # Add a 'priorsurg' column with binary values (0 or 1)
# # X['priorsurg'] = np.random.choice([0, 1], len(X))
# X['priorsurg'] = np.random.choice(['0', '1', True, False], len(X))

# # Display the first few rows of the messy dataset
# print(X.head())

# # Display the metadata and variable information
# print(heart_disease.metadata)
# print(heart_disease.variables)


   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

    ca  thal  
0  0.0   6.0  
1  3.0   3.0  
2  2.0   7.0  
3  0.0   3.0  
4  0.0   3.0  
   num
0    0
1    2
2    1
3    0
4    0


In [9]:
heart_disease.keys()
heart_disease['variables']


,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,years,no
1,sex,Feature,Categorical,Sex,None,None,no
2,cp,Feature,Categorical,None,None,None,no
3,trestbps,Feature,Integer,None,resting blood pressure (on admission to the ho...,mm Hg,no
4,chol,Feature,Integer,None,serum cholestoral,mg/dl,no
5,fbs,Feature,Categorical,None,fasting blood sugar > 120 mg/dl,None,no
6,restecg,Feature,Categorical,None,None,None,no
7,thalach,Feature,Integer,None,maximum heart rate achieved,None,no
8,exang,Feature,Categorical,None,exercise induced angina,None,no
9,oldpeak,Feature,Integer,None,ST depression induced by exercise relative to ...,None,no


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Encode categorical variables
# label_encoders = {}
# for column in X.select_dtypes(include=['object']).columns:
# 	le = LabelEncoder()
# 	X[column] = le.fit_transform(X[column])
# 	label_encoders[column] = le

# remove any rows with missing values from both X and y
missing = X.isnull().any(axis=1)
X = X[~missing]
y = y[~missing]

# binarize y values
y['num'] = y['num'].apply(lambda x: 1 if x > 0 else 0)

# Split the data into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y['num'], test_size=0.2, stratify=y['num'], random_state=42)

# Initialize the logistic regression model
log_reg = LogisticRegression(random_state=42, max_iter=1000, solver='liblinear', penalty='l2', C=1.0)

# Train the model
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

/var/folders/k9/vcw_mvv16nxc8ncjdxnvv5fm0000gn/T/ipykernel_14860/3799730800.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['num'] = y['num'].apply(lambda x: 1 if x > 0 else 0)


ValueError: could not convert string to float: 'ls3'

In [24]:
classification_report?

Signature:
classification_report(
    y_true,
    y_pred,
    *,
    labels=None,
    target_names=None,
    sample_weight=None,
    digits=2,
    output_dict=False,
    zero_division='warn',
)
Docstring:
Build a text report showing the main classification metrics.

Read more in the :ref:`User Guide <classification_report>`.

Parameters
----------
y_true : 1d array-like, or label indicator array / sparse matrix
    Ground truth (correct) target values.

y_pred : 1d array-like, or label indicator array / sparse matrix
    Estimated targets as returned by a classifier.

labels : array-like of shape (n_labels,), default=None
    Optional list of label indices to include in the report.

target_names : array-like of shape (n_labels,), default=None
    Optional display names matching the labels (same order).

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights.

digits : int, default=2
    Number of digits for formatting output floating point values.
    When ``

In [ ]:
# # make the data messy
# # data['sex'] = data['sex'].apply(lambda x: np.random.choice(['M', 'MALE', 'Male']) if x == 'male' else np.random.choice(['F', 'FEMALE', 'Female']))
# # data.pop('deck')
# data.pop('alive')

# # duplicate a random numerical column and inject outliers
# data['wspd'] = data['age'].apply(lambda x: x + np.random.choice([0, 10, 200]) if np.random.choice([0, 1]) == 1 else x)

# # introduce redundant columns
# data['tprc'] = data['fare'] * 3 + np.random.choice([0, 1, 2])
# data['lfwa'] = data['age'] * 2 + np.random.choice([0, 1, 2])

# # randomly introduce missing values in numerical columns
# for col in ['age', 'fare', 'pclass', 'sibsp', 'parch']:
#     data[col] = data[col].apply(lambda x: np.nan if np.random.choice([0, 1], p=[0.9, 0.1]) == 1 else x)

# # add duplicates (last thing to do)
# data = pd.concat([data, data.sample(frac=0.3, replace=True, random_state=42)], axis=0)

# # shuffle the columns 
# cols = data.columns[1:].tolist()
# np.random.shuffle(cols)
# data = data[['survived'] + cols]

In [ ]:
# Identify missing values
missing_data = data.isnull().sum()
print(missing_data)

# Impute missing values
data['age'].fillna(data['age'].mean(), inplace=True)
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)

# Drop rows with missing values
data.dropna(inplace=True)

# Display the dataset after handling missing data
data.head()
